In [ ]:
from transformers import (
    Qwen2_5_VLForConditionalGeneration,
    AutoTokenizer,
    AutoProcessor,
)
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-3B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|im_end|>
<|im_start|>assistant



In [ ]:
image_inputs, video_inputs = process_vision_info(messages)
print(f"{image_inputs=}")
print(f"{video_inputs=}")

image_inputs=[<PIL.Image.Image image mode=RGB size=2044x1372 at 0x7F26591F8DD0>]
video_inputs=None


In [ ]:
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")
for k, v in inputs.items():
    print(f"{k:<10}, {str(v.shape):<10}")

input_ids , torch.Size([1, 3602])
attention_mask, torch.Size([1, 3602])
pixel_values, torch.Size([14308, 1176])
image_grid_thw, torch.Size([1, 3])


In [ ]:
# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['The image depicts a serene beach scene with a person and a dog. The person is sitting on the sandy beach, facing the ocean, and appears to be interacting with the dog. The dog is also sitting on the sand, facing the person, and seems to be giving a paw or a friendly gesture. The person is wearing a plaid shirt and has long hair. The background shows the ocean with gentle waves and a clear sky, suggesting it might be early morning or late afternoon due to the soft lighting. The overall atmosphere of the image is calm and joyful.']
